In [1]:
import pandas as pd
from datetime import datetime
import time
import numpy as np
import re
import datetime

import tensorflow as tf

C:\Python\Anaconda3-52\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('./team_project_2/basic_dataset/SanFrancisco_Crime_Classification/tmp_theft.csv',encoding = 'utf-8')

In [3]:
del df["Unnamed: 0"]

In [4]:
# 가로, 세로를 몇 개로 나눌지 x_n, y_n만 바꾸면 됨
x_n=5
y_n=5
X_category=[]
for i in range(x_n+1):
    x_category = min(df['X'])+ ((max(df['X'])-min(df['X']))/x_n)*(i)
    X_category.append(x_category)
print("X_category:",X_category)

Y_category=[]
for i in range(y_n+1):
    y_category = min(df['Y'])+ ((max(df['Y'])-min(df['Y']))/y_n)*(i)
    Y_category.append(y_category)
print("Y_category:",Y_category)

X_category: [-122.51364206429, -122.110913651432, -121.708185238574, -121.30545682571599, -120.902728412858, -120.5]
Y_category: [37.7079683645097, 48.166374691607764, 58.62478101870582, 69.08318734580388, 79.54159367290194, 90.0]


In [5]:
X=[]
for idx in range(len(df['X'])):
    if df['X'][idx] <= X_category[1]:
        X.append((X_category[1]+X_category[0])/2)
        continue
    if df['X'][idx] <= X_category[2]:
        X.append((X_category[2]+X_category[1])/2)
        continue
    if df['X'][idx] <= X_category[3]:
        X.append((X_category[3]+X_category[2])/2)
        continue
    if df['X'][idx] <= X_category[4]:
        X.append((X_category[4]+X_category[3])/2) 
        continue
    else:
        X.append((X_category[5]+X_category[4])/2)        
len(X)

174900

In [6]:
Y=[]
for idx in range(len(df['Y'])):
    if df['Y'][idx] <= Y_category[1]:
        Y.append((Y_category[1]+Y_category[0])/2)
        continue
    if df['Y'][idx] <= Y_category[2]:
        Y.append((Y_category[2]+Y_category[1])/2)
        continue
    if df['Y'][idx] <= Y_category[3]:
        Y.append((Y_category[3]+Y_category[2])/2)
        continue
    if df['Y'][idx] <= Y_category[4]:
        Y.append((Y_category[4]+Y_category[3])/2) 
        continue
    else:
        Y.append((Y_category[5]+Y_category[4])/2)
len(Y)

174900

In [7]:
df_X=pd.DataFrame(X).rename(columns={0:'X_group'})
df_Y=pd.DataFrame(Y).rename(columns={0:'Y_group'})

In [8]:
df_crime= pd.concat([df, df_X, df_Y], axis=1)
df_crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Date,...,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,X_group,Y_group
0,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015-05-13 23:30:00,...,0,0,1,0,0,0,0,0,-122.312278,42.937172
1,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015-05-13 23:30:00,...,0,0,0,1,0,0,0,0,-122.312278,42.937172
2,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431,2015-05-13 23:30:00,...,1,0,0,0,0,0,0,0,-122.312278,42.937172
3,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601,2015-05-13 23:00:00,...,0,0,0,0,1,0,0,0,-122.312278,42.937172
4,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802,2015-05-13 23:00:00,...,0,0,0,0,0,0,0,0,-122.312278,42.937172


In [9]:
del df_crime['Category_index']

In [10]:
df_crime.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Date', 'Year', 'Month', 'Day',
       'Hour', 'DayOfWeek_index', 'PdDistrict_index', 'Descript_index',
       'Resolution_index', 'Address_index', 'Friday', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'BAYVIEW', 'CENTRAL',
       'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN',
       'TARAVAL', 'TENDERLOIN', 'X_group', 'Y_group'],
      dtype='object')

In [11]:
df_crime =df_crime[['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Date', 'Year', 'Month', 'Day', 'DayOfWeek_index', 'PdDistrict_index',
       'Descript_index', 'Resolution_index', 'Address_index', 
       'Monday', 'Tuesday', 'Wednesday','Thursday', 'Friday','Saturday', 'Sunday', 
       'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK',
       'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 'X_group', 'Y_group',
       'Hour']]

In [12]:
df_crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Date,...,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,X_group,Y_group,Hour
0,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015-05-13 23:30:00,...,0,1,0,0,0,0,0,-122.312278,42.937172,23
1,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015-05-13 23:30:00,...,0,0,1,0,0,0,0,-122.312278,42.937172,23
2,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431,2015-05-13 23:30:00,...,0,0,0,0,0,0,0,-122.312278,42.937172,23
3,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601,2015-05-13 23:00:00,...,0,0,0,1,0,0,0,-122.312278,42.937172,23
4,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802,2015-05-13 23:00:00,...,0,0,0,0,0,0,0,-122.312278,42.937172,23


In [13]:
df_crime.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Date', 'Year', 'Month', 'Day',
       'DayOfWeek_index', 'PdDistrict_index', 'Descript_index',
       'Resolution_index', 'Address_index', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday', 'BAYVIEW', 'CENTRAL',
       'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN',
       'TARAVAL', 'TENDERLOIN', 'X_group', 'Y_group', 'Hour'],
      dtype='object')

In [14]:
min(df_crime['X_group']),max(df_crime['X_group'])

(-122.312277857861, -120.70136420642899)

In [15]:
# df_crime['Wednesday']

In [16]:
df_crime.to_csv('./team_project_2/basic_dataset/SanFrancisco_Crime_Classification/theft_dataset.csv',sep=',', encoding='utf-8')

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_crime, test_size=0.275)

In [18]:
len(train), len(test)

(126802, 48098)

In [19]:
df_x =df_crime[['X', 'Y', 'Year', 'Month', 'Day', 'DayOfWeek_index', 'PdDistrict_index',
       'Descript_index', 'Resolution_index', 'Address_index', 
       'Monday', 'Tuesday', 'Wednesday','Thursday', 'Friday','Saturday', 'Sunday', 
       'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK',
       'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 'X_group', 'Y_group']]

In [20]:
len(df_x.columns)

29

In [21]:
df_y =df_crime[['Hour']]

In [22]:
from tqdm import tqdm_notebook
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

In [23]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [24]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.reset_default_graph()

x_data = df_x
sess = tf.Session()
y_data = tf.one_hot(df_y, depth = 24).eval(session=sess)
y_data = tf.reshape(y_data, shape=[-1,24]).eval(session=sess)
print(y_data)
tf.set_random_seed(1004)  # reproducibility



# parameters
learning_rate = 0.0001



X = tf.placeholder(tf.float32, [None, 29])
Y = tf.placeholder(tf.float32, [None, 24])

W1 = tf.get_variable("W1", shape=[29, 64],
                     initializer=xavier_init(29, 64))
b1 = tf.Variable(tf.random_normal([64]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[64, 64],
                     initializer=xavier_init(64, 64))
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[64, 24],
                     initializer=xavier_init(64, 24))
b3 = tf.Variable(tf.random_normal([24]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(10001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 1000 == 0 or step < 100:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}, \t Cost: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Cost: 1125.713, 	 Acc: 1.08%
Step:     1, 	 Cost: 1112.940, 	 Acc: 1.08%
Step:     2, 	 Cost: 1100.196, 	 Acc: 1.08%
Step:     3, 	 Cost: 1087.545, 	 Acc: 1.07%
Step:     4, 	 Cost: 1075.041, 	 Acc: 1.07%
Step:     5, 	 Cost: 1062.719, 	 Acc: 1.07%
Step:     6, 	 Cost: 1050.515, 	 Acc: 1.06%
Step:     7, 	 Cost: 1038.387, 	 Acc: 1.06%
Step:     8, 	 Cost: 1026.330, 	 Acc: 1.05%
Step:     9, 	 Cost: 1014.339, 	 Acc: 1.05%
Step:    10, 	 Cost: 1002.399, 	 Acc: 1.04%
Step:    11, 	 Cost: 990.492, 	 Acc: 1.05%
Step:    12, 	 Cost: 978.634, 	 Acc: 1.06%
Step:    13, 	 Cost: 966.840, 	 Acc: 1.09%
Step:    14, 	 Cost: 955.130, 	 Acc: 1.11%
Step:    15, 	 Cost: 943.495, 	 Acc: 1.14%
Step:    16, 	 Cost: 931.993, 	 Acc: 1.33%
Step:    17, 	 Cost: 920.662, 	 Acc: 1.53%
Step:    18, 	 Cost: 909.485, 	 Acc: 1.68%
Step:    19, 	 Cost: 898.413, 	 Acc: 1.78%
Step:    20, 	 Cost: 887.437, 	 Acc: 1.80%
Step:    21, 	 Cost: 876.662, 	 Acc: 1.94%
Step:    22, 	 Cost: 866.221, 	 Acc: 2.12%
